In [18]:
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import Word2Vec, CoherenceModel
from gensim.models.keyedvectors import KeyedVectors
import mlflow, os, nltk, random, re
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
import numpy as np
from nltk.tokenize import sent_tokenize, word_tokenize
from collections import Counter
import pyLDAvis
import pyLDAvis.gensim
import nltk
from nltk import pos_tag
from nltk.corpus import wordnet, stopwords, words
from nltk.stem import WordNetLemmatizer
from pprint import pprint

In [19]:
nltk.download('words')
nltk.download('punkt')
nltk.download('wordnet')

stop_words = stopwords.words('english')
vocabulary = words.words()

[nltk_data] Downloading package words to
[nltk_data]     /Users/thanawatthongpia/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/thanawatthongpia/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/thanawatthongpia/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [20]:
# Convert GloVe vectors to word2vec format
# from gensim.scripts.glove2word2vec import glove2word2vec

# glove_file = "src/glove.42B.300d.txt"
# temp_word2vec_file = "src/w2v.42B.300d.txt"
# glove2word2vec(glove_file, temp_word2vec_file)

In [21]:
def preprocess_text(sentences):

    lemmatizer = WordNetLemmatizer()
    lemmatized_sentences = []
    bigram = gensim.models.Phrases(sentences, min_count=5, threshold=100)
    bigram_mod = gensim.models.phrases.Phraser(bigram)
    
    data_words_nostops = [[word for word in doc if word not in stop_words] for doc in sentences]
    data_words_bigrams = [bigram_mod[doc] for doc in data_words_nostops]

    for sentence in data_words_bigrams:
        lemmatized_tokens = [lemmatizer.lemmatize(token, wordnet.VERB) for token in sentence]
        lemmatized_sentences.append(lemmatized_tokens)
    return lemmatized_sentences

In [22]:
def import_text_to_list_of_lists(file_path, delimiter='\t'):
    result = []
    with open(file_path, 'r') as file:
        for line in file:
            line = line.strip()  # Remove leading/trailing whitespace and newlines
            sublist = line.split(delimiter)
            result.append(sublist)
    return result

In [23]:
def CreateW2V_Model(w2v_file_name, pre_train_file, vectorsize=300, window=5, min_count=1, workers=4):

    folder_path = 'src'
    pre_train_file_path = os.path.join(folder_path, pre_train_file)
    w2v_file_path = os.path.join(folder_path, w2v_file_name)
    RUN_NAME = "Create Word2Vec Model"

    if mlflow.active_run():
        mlflow.end_run()
    
    with mlflow.start_run(run_name=RUN_NAME):
        # Check exist model
        if os.path.isfile(pre_train_file_path):
            # Load word vectors
            GloVe_vectors = KeyedVectors.load(pre_train_file_path)
        else:
            GloVe_vectors = KeyedVectors.load_word2vec_format(w2v_file_path, binary=False)

        # Initialize a Gensim Word2Vec model
        model = Word2Vec(vector_size=vectorsize, window=window, min_count=min_count, workers=workers)
        # Set the vocabulary and vectors of the model
        model.build_vocab_from_freq(GloVe_vectors.key_to_index)
        model.wv.key_to_index = GloVe_vectors.key_to_index
        model.wv.vectors = GloVe_vectors.vectors
        model.wv.index_to_key = GloVe_vectors.index_to_key
        
        if not os.path.isfile(pre_train_file_path):
            model.wv.save(pre_train_file_path) 

        # Log parameters
        mlflow.log_param("vectorsize", vectorsize)
        mlflow.log_param("window", window)
        mlflow.log_param("min_count", min_count)
        mlflow.log_param("workers", workers)

        # Log model artifact
        mlflow.log_artifact(pre_train_file_path)
        
        return model

In [24]:
def test_Accuracy(model , test_file_path):
    with open(test_file_path, 'r') as file:
        lines = file.readlines()
    total_questions = 0
    correct_predictions = 0
    random_amount = 1000
    random.seed(42)
    for line in random.sample(lines, random_amount):
        if line.startswith(':'):
            continue  # Skip comment lines and category headers
        question, expected_answer = [word.lower() for word in line.split()[:2]]
        choices = [word.lower() for word in line.split()[2:]]
        try:
            predicted_answer = model.wv.most_similar(positive=[question, choices[1]], negative=[expected_answer])[0][0]
            if predicted_answer == choices[0]:
                correct_predictions += 1
        except KeyError:
            continue  # Skip questions with out-of-vocabulary words

        total_questions += 1

    accuracy = correct_predictions / total_questions * 100
    return accuracy

In [25]:
def train_with_corpus(model, RUN_NAME, new_corpus, epochs):
    
    if mlflow.active_run():
        mlflow.end_run()
        
    with mlflow.start_run(run_name=RUN_NAME):
        # Update the vocabulary with new words
        model.build_vocab(new_corpus, update=True)
        model.train(new_corpus, total_examples=len(new_corpus), epochs=epochs)

        # Log parameters
        mlflow.log_param("epochs", epochs)
        mlflow.log_param("train round", i)
        # Log model artifact
        # mlflow.sklearn.log_model(model, "word2vec_model")

        accuracy = test_Accuracy(model,'src/questions-words.txt' )
        mlflow.log_metric("accuracy", accuracy)
        return model

In [26]:
def plot_vec(model):
    
    vocabulary = words.words()
    all_words = []
    vectors = []
    vocabs = list(set(vocabulary) & set(model.wv.key_to_index.keys()))
    random.seed(42)
    random_amount = 100
    for word in random.sample(vocabs, random_amount):
        all_words.append(word)
        vectors.append(model.wv[word])
        
    tsne = TSNE(n_components=2, random_state=42)
    vectors_tsne = tsne.fit_transform(np.array(vectors))

    #########   PLOT CHART    #######
    plt.figure(figsize=(10, 10))
    for i, word in enumerate(all_words):
        x, y = vectors_tsne[i]
        plt.scatter(x, y)
        plt.annotate(word, (x, y), alpha=0.5, fontsize=8)

    plt.title("t-SNE Visualization of Word2Vec Embeddings")
    plt.show()

In [27]:
def corpus_topic(corpus,num_topics=10):

    data_lemmatized = preprocess_text(corpus)
    id2word = corpora.Dictionary(data_lemmatized)  

    # Term Document Frequency 
    LDA_corpus = [id2word.doc2bow(text) for text in data_lemmatized]
    lda_model = gensim.models.ldamodel.LdaModel(corpus=LDA_corpus,
                                           id2word=id2word,
                                           num_topics=num_topics, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)
    
    # Compute Perplexity
    print('\nPerplexity: ', lda_model.log_perplexity(LDA_corpus))  
    # a measure of how good the model is. lower the better.

    # Compute Coherence Score
    coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
    coherence_lda = coherence_model_lda.get_coherence()
    print('Coherence Score: ', coherence_lda)
    pyLDAvis.enable_notebook()
    vis = pyLDAvis.gensim.prepare(lda_model, LDA_corpus, id2word)
    return vis

In [28]:
model = CreateW2V_Model("w2v.42B.300d.txt",'GloVe.wordvectors')

In [29]:
model.wv.most_similar("apple", topn=5)

[('iphone', 0.7339416146278381),
 ('ipad', 0.7129027247428894),
 ('apples', 0.6433200836181641),
 ('blackberry', 0.6380701661109924),
 ('ipod', 0.6341225504875183)]

In [30]:
# for time in range(5):
#     model = train_with_corpus(model,new_corpus,1000)
#     i+=1
#     model.wv.most_similar("apple", topn=5)

In [31]:
# plot_vec(model)

In [32]:
# test_Accuracy(model,'src/questions-words.txt' )

In [33]:
# scb_corpus = import_text_to_list_of_lists('src/SCB_corpus_tip4u.txt', delimiter='\t')
techx_corpus = import_text_to_list_of_lists('src/TechX_corpus.txt', delimiter='\t')
pointx_corpus = import_text_to_list_of_lists('src/PointX_corpus.txt', delimiter='\t')

In [34]:
key = 'scb'
try:
    print(model.wv.most_similar(key,topn=5))
except KeyError:
    print(f"{key} not found")

[('aic', 0.3304489850997925), ('gsc', 0.32839545607566833), ('m&i', 0.3262021243572235), ('laurentian', 0.32590141892433167), ('iob', 0.3192133605480194)]


In [35]:
# don't know why preprocess techx corpus first then kernal will crashed
train_datasets = [preprocess_text(dataset) for dataset in [pointx_corpus, techx_corpus]]

for i,dataset in enumerate(train_datasets):
    runname = f"train round {i+1}"
    TechXmodel = train_with_corpus(model,runname,dataset,epochs=100)

In [36]:
try:
    print(TechXmodel.wv.most_similar(key,topn=5))
except KeyError:
    print(f"{key} not found")

[('cardx', 0.9924686551094055), ('call_center', 0.9924042224884033), ('please_contact', 0.9923171997070312), ('facebook_linkedin', 0.9923133850097656), ('anddigital', 0.9923035502433777)]


In [37]:
corpus_topic(preprocess_text(techx_corpus))


Perplexity:  -6.684810601212469
Coherence Score:  0.43309610510121493


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
5     -0.290689  0.108397       1        1  16.719114
7     -0.305277  0.027804       2        1  13.192491
2     -0.045961 -0.398084       3        1  12.570399
1      0.058823  0.019101       4        1  12.434667
3      0.043349  0.151301       5        1  10.949949
8      0.149864  0.014750       6        1  10.374636
9      0.133667  0.043595       7        1   8.851225
6      0.133800  0.048923       8        1   6.328677
0      0.047617 -0.022675       9        1   4.967797
4      0.074806  0.006887      10        1   3.611044, topic_info=          Term        Freq       Total Category  logprob  loglift
1132     point  777.000000  777.000000  Default  30.0000  30.0000
18         scb  609.000000  609.000000  Default  29.0000  29.0000
869     pointx  811.000000  811.000000  Default  28.0000  28.0000
834      techx  363.000000  363.000000  Default  27.0000  27.0000
448    digital  295.000000  295.000000  Default  26.0000  26.0000
...        ...         ...         ...      ...      ...      ...
1782     roles    5.750574    6.703337  Topic10  -5.1734   3.1679
1907     phase    5.732428    6.684995  Topic10  -5.1766   3.1674
75     process   15.838060   22.745829  Topic10  -4.1603   2.9592
115     credit   38.522693   89.019052  Topic10  -3.2715   2.4836
858   solution   10.681509   34.842405  Topic10  -4.5542   2.1389

[399 rows x 6 columns], token_table=      Topic      Freq          Term
term                               
149       8  0.946366          able
200       6  0.927438       account
1747      1  0.981170    accumulate
1124      2  0.987829  accumulation
1152      4  0.559325        across
...     ...       ...           ...
616       8  0.330556          work
616       9  0.157408          work
1151      2  0.991535         world
1681      4  0.952868          year
1304      3  0.959494         years

[415 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[6, 8, 3, 2, 4, 9, 10, 7, 1, 5])

In [38]:
corpus_topic(preprocess_text(pointx_corpus))


Perplexity:  -5.002372597595141
Coherence Score:  0.6499014154296792


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0     -0.240995  0.122344       1        1  24.804117
2      0.161659  0.111441       2        1  15.054724
1     -0.160022 -0.223341       3        1  11.417325
4     -0.002594  0.128302       4        1  10.357153
9      0.287387  0.002685       5        1   9.180922
8      0.033690 -0.182296       6        1   8.844756
5     -0.131510  0.152089       7        1   8.638766
6     -0.064550 -0.114218       8        1   4.546301
7      0.127443 -0.042826       9        1   4.047838
3     -0.010508  0.045822      10        1   3.108097, topic_info=        Term        Freq       Total Category  logprob  loglift
23   special   82.000000   82.000000  Default  30.0000  30.0000
70     score   68.000000   68.000000  Default  29.0000  29.0000
232     june   55.000000   55.000000  Default  28.0000  28.0000
42     point  196.000000  196.000000  Default  27.0000  27.0000
50       use   56.000000   56.000000  Default  26.0000  26.0000
..       ...         ...         ...      ...      ...      ...
69   receive    2.622566   64.203375  Topic10  -3.4229   0.2733
70     score    1.116896   68.990352  Topic10  -4.2765  -0.6523
42     point    0.932262  196.740416  Topic10  -4.4572  -1.8809
215     line    0.368023    7.393809  Topic10  -5.3867   0.4709
216      man    0.368023    7.073225  Topic10  -5.3867   0.5152

[407 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
227       1  0.630261      accord
116       4  0.937554  accordance
107       1  0.347427     account
107       2  0.138971     account
107      10  0.416913     account
...     ...       ...         ...
272       3  0.782119   wednesday
311       1  0.735850        week
131       4  0.531570      within
52        6  0.826486       world
224       3  0.949549        year

[367 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 3, 2, 5, 10, 9, 6, 7, 8, 4])

In [39]:
TechXmodel.wv.most_similar('techx',topn=10)

[('consumer_demands', 0.9919575452804565),
 ('institutions_reinvent', 0.9919145107269287),
 ('stay_ahead', 0.991886556148529),
 ('cooperative_education', 0.991872251033783),
 ('trirat_suwanprateeb', 0.9917877316474915),
 ('focused_subsidiary', 0.9917343854904175),
 ('scbx', 0.9917099475860596),
 ('southeast_asias', 0.991675615310669),
 ('technological_capabilities', 0.9916498064994812),
 ('facebook_linkedin', 0.991629958152771)]